In [1]:
import pymysql
import pandas as pd

conn = pymysql.connect(host='localhost', port = 3306, user='root', 
            password='1234', db='INVESTAR', charset='utf8')
sql = "SELECT * FROM test_data"
df = pd.read_sql(sql, conn)
df.head()

,Direction,Col1,Col2,Result
0,하락형,D1,D1,하락
1,상승형,U1,U1,상승
2,하락형,D5,D6,하락
3,하락형,U8,D10,상승
4,상승형,E2,U2,하락


In [3]:
x = df.drop(['Result'], axis = 1)
x = pd.get_dummies(x, drop_first = True)
y = df['Result']
y.value_counts()

보합    7105
상승    4957
하락    4560
Name: Result, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle=True,random_state=1234)

In [14]:
# SVM - linearSVC
from sklearn.svm import LinearSVC
C = 0.1
lsvc_clf = LinearSVC(C=C, max_iter = 10000, multi_class = 'ovr')
lsvc_clf.fit(x_train,y_train)
y_pred = lsvc_clf.predict(x_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.49297971918876754


In [15]:
# svm - svc
from sklearn.svm import SVC
svc_clf = SVC(kernel = 'rbf', probability = True)
svc_clf.fit(x_train,y_train)
y_pred = svc_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.48798751950078


In [7]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.4237129485179407


In [9]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier()
params = {'max_depth' : [6,8,10,12,16,20,24], 'min_samples_split' : [16, 24]}
grid_cv = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1)
grid_cv.fit(x_train, y_train)
print('최고 평균 정확도 수치 : {0:4f}'.format(grid_cv.best_score_))
print('최적 하이퍼 파라미터 :', grid_cv.best_params_)

best_df_clf = grid_cv.best_estimator_
pred1 = best_df_clf.predict(x_test)
accuracy = accuracy_score(y_test, pred1)
print('결정트리 예측 정확도 : {:.4f}'.format(accuracy))
dt_clf = DecisionTreeClassifier(max_depth = 12, min_samples_split = 16)
dt_clf.fit(x_train, y_train)
pred = dt_clf.predict(x_test)
accuracy = accuracy_score(y_test, pred)
print(accuracy)

Fitting 5 folds for each of 14 candidates, totalling 70 fits
최고 평균 정확도 수치 : 0.493119
최적 하이퍼 파라미터 : {'max_depth': 12, 'min_samples_split': 16}
결정트리 예측 정확도 : 0.4758
0.47578947368421054


In [16]:
# VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
rnd_clf = RandomForestClassifier()
voting_clf = VotingClassifier(estimators = [('rf', rnd_clf), ('svc', svc_clf)],
                             voting = 'soft')
voting_clf.fit(x_train, y_train)
for clf in (rnd_clf, svc_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

RandomForestClassifier 0.48798751950078
SVC 0.48798751950078
VotingClassifier 0.48985959438377535


In [20]:
# Bagging
from sklearn.ensemble import BaggingClassifier
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                           max_samples = 10000, bootstrap=True, n_jobs=-1, oob_score = True)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.4901716068642746


In [19]:
# Pasting
past_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                           max_samples = 300, bootstrap=False, n_jobs=-1)
past_clf.fit(x_train, y_train)
y_pred = past_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.49297971918876754


In [27]:
# adaboost boosting
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 2), n_estimators = 200,
                           algorithm='SAMME.R', learning_rate = 0.5)
ada_clf.fit(x_train, y_train)
y_pred = ada_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

0.49890795631825274


In [26]:
# gradient boosting
import xgboost
xgb_clf = xgboost.XGBClassifier()
xgb_clf.fit(x_train, y_train, eval_set = [(x_test, y_test)], early_stopping_rounds=2)
y_pred = xgb_clf.predict(x_test)
print(accuracy_score(y_test, y_pred))

[14:36:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-mlogloss:1.06610
[1]	validation_0-mlogloss:1.04581
[2]	validation_0-mlogloss:1.03179
[3]	validation_0-mlogloss:1.02254


C:\Users\82106\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[4]	validation_0-mlogloss:1.01569
[5]	validation_0-mlogloss:1.01086
[6]	validation_0-mlogloss:1.00763
[7]	validation_0-mlogloss:1.00514
[8]	validation_0-mlogloss:1.00322
[9]	validation_0-mlogloss:1.00215
[10]	validation_0-mlogloss:1.00089
[11]	validation_0-mlogloss:1.00007
[12]	validation_0-mlogloss:0.99911
[13]	validation_0-mlogloss:0.99825
[14]	validation_0-mlogloss:0.99757
[15]	validation_0-mlogloss:0.99713
[16]	validation_0-mlogloss:0.99659
[17]	validation_0-mlogloss:0.99643
[18]	validation_0-mlogloss:0.99615
[19]	validation_0-mlogloss:0.99554
[20]	validation_0-mlogloss:0.99532
[21]	validation_0-mlogloss:0.99540
[22]	validation_0-mlogloss:0.99487
[23]	validation_0-mlogloss:0.99474
[24]	validation_0-mlogloss:0.99485
[25]	validation_0-mlogloss:0.99469
[26]	validation_0-mlogloss:0.99456
[27]	validation_0-mlogloss:0.99455
[28]	validation_0-mlogloss:0.99447
[29]	validation_0-mlogloss:0.99458
[30]	validation_0-mlogloss:0.99438
[31]	validation_0-mlogloss:0.99425
[32]	validation_0-mlogloss

In [11]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from sklearn.metrics import accuracy_score

# SVM - linearSVC
C = 0.1
lsvc_clf = LinearSVC(C=C, max_iter = 10000, multi_class = 'ovr')
lsvc_clf.fit(x_train,y_train)
y_pred = lsvc_clf.predict(x_test)
print(f'LinearSVD 정확도 : {accuracy_score(y_test, y_pred)}')

# svm - svc
svc_clf = SVC(kernel = 'rbf', probability = True)
svc_clf.fit(x_train,y_train)
y_pred = svc_clf.predict(x_test)
print(f'SVC 정확도 : {accuracy_score(y_test, y_pred)}')

#KNN
knn = KNeighborsClassifier(n_neighbors = 6)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(f'KNN 정확도 : {accuracy_score(y_test, y_pred)}')

# Decision Tree
dt = DecisionTreeClassifier(random_state=156)
dt.fit(x_train, y_train)
y_pred = dt.predict(x_test)
print(f'Decision Tree : {accuracy_score(y_test, y_pred)}')

# VotingClassifier
print('Voting')
rnd_clf = RandomForestClassifier()
voting_clf = VotingClassifier(estimators = [('rf', rnd_clf), ('svc', svc_clf)],
                             voting = 'soft')
voting_clf.fit(x_train, y_train)
for clf in (rnd_clf, svc_clf, voting_clf):
    clf.fit(x_train, y_train)
    y_pred = clf.predict(x_test)
    print(' ',clf.__class__.__name__,'정확도 :', accuracy_score(y_test, y_pred))
    
# Bagging
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                           max_samples = 10000, bootstrap=True, n_jobs=-1, oob_score = True)
bag_clf.fit(x_train, y_train)
y_pred = bag_clf.predict(x_test)
print(f'Bagging : {accuracy_score(y_test, y_pred)}')

# Pasting
past_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 500,
                           max_samples = 300, bootstrap=False, n_jobs=-1)
past_clf.fit(x_train, y_train)
y_pred = past_clf.predict(x_test)
print(f'Pasting : {accuracy_score(y_test, y_pred)}')

# adaboost boosting
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 2), n_estimators = 200,
                           algorithm='SAMME.R', learning_rate = 0.5)
ada_clf.fit(x_train, y_train)
y_pred = ada_clf.predict(x_test)
print(f'adaboost boosting : {accuracy_score(y_test, y_pred)}')

LinearSVD 정확도 : 0.47729323308270677
SVC 정확도 : 0.48
KNN 정확도 : 0.4336842105263158
Decision Tree : 0.4724812030075188
Voting
  RandomForestClassifier 정확도 : 0.4733834586466165
  SVC 정확도 : 0.48
  VotingClassifier 정확도 : 0.47458646616541356
Bagging : 0.47398496240601506
Pasting : 0.4766917293233083
adaboost boosting : 0.4781954887218045
